In [1]:
import requests
import json
import pandas as pd
import numpy as np
from pathlib import Path
from collections import Counter

In [2]:
%load_ext autoreload
%autoreload 1
%aimport pantheon_statistics
%aimport pantheon_data

In [3]:
from pantheon_statistics import process_game, player_statistics_dict, add_difference, get_dealer_id, \
    load_games_from_file, load_participating_players_from_file, load_games_from_mjtop, load_participating_players_from_mjtop
from pantheon_data import yaku_map_eng, yaku_map_short, yaku_eng_rus, yaku_map_rus, yaku_short_rus

In [4]:
event_id = 204

In [5]:
store_folder = Path('data/')
games_json = store_folder / f'games_id{event_id}.json'
participating_players_json = store_folder / f'participating_players_id{event_id}.json'
if not games_json.is_file() and not participating_players_json.is_file():
    games = load_games_from_mjtop(event_id)
    participating_players = load_participating_players_from_mjtop(event_id)
else:
    games = load_games_from_file(games_json)
    participating_players = load_participating_players_from_file(participating_players_json)

In [6]:
# Сохранить на диск для работы в офлайн-режиме
# При наличии этих файлов новая статистика с Пантеона не загружается
if not store_folder.exists():
    store_folder.mkdir()
with open(games_json, 'w', encoding='utf-8') as outfile:
    json.dump(games, outfile, ensure_ascii=False, indent='  ')
with open(participating_players_json, 'w', encoding='utf-8') as outfile:
    json.dump(participating_players, outfile, ensure_ascii=False, indent='  ')

In [7]:
games[1]['final_results']

{'770': {'score': 52800, 'rating_delta': 37800, 'place': 1},
 '36': {'score': 25800, 'rating_delta': -9200, 'place': 3},
 '388': {'score': 13400, 'rating_delta': -31600, 'place': 4},
 '315': {'score': 28000, 'rating_delta': 3000, 'place': 2}}

In [8]:
participating_players[1]

{'id': 12,
 'display_name': 'Мешин Виталий',
 'alias': 'meshin',
 'local_id': None,
 'team_name': None,
 'tenhou_id': 'Benawi'}

In [9]:
players = {}
for game in games:
    diffs = process_game(game, participating_players)
    for player_id in diffs:
        if player_id not in players:
            players[player_id] = player_statistics_dict(player_id, participating_players)
        add_difference(
            base=players[player_id],
            diff=diffs[player_id])   

players;
players[316]

{'player_id': 316,
 'player_name': 'Мишаков Дмитрий',
 'player_team': None,
 'player_tenhou_id': 'NTshavic',
 'num_games': 7,
 'num_games_aborted': 0,
 'num_rounds': 83,
 'num_rounds_dealer': 21,
 'num_dealer_wins': 6,
 'num_wins': 19,
 'num_wins_ron': 11,
 'num_wins_double_ron': 0,
 'num_wins_triple_ron': 0,
 'num_wins_tsumo': 8,
 'num_wins_open_hand': 8,
 'num_draws': 10,
 'num_draws_tempai': 8,
 'num_draws_noten': 2,
 'num_abort': 0,
 'num_chombo': 0,
 'num_feeds': 12,
 'num_feeds_under_riichi': 2,
 'num_tsumo_feeds': 12,
 'num_tsumo_feeds_dealer': 1,
 'riichi_bets_wins': 7,
 'riichi_bets_lost': 8,
 'doras': 33,
 'doras_normal': 33,
 'doras_uradora': 0,
 'doras_kandora': 0,
 'doras_kanuradora': 0,
 'doras_akadora': 0,
 'gains_ron': 88100,
 'gains_tsumo': 63700,
 'gains_ron_dealer': 33400,
 'gains_tsumo_dealer': 31300,
 'gains_tempai': 13000,
 'losses_ron': 89600,
 'losses_tsumo': 21500,
 'losses_tsumo_dealer': 4100,
 'losses_noten': 2500,
 'yaku': {23: 6, 33: 10, 13: 3, 35: 1, 31: 1

In [10]:
data = {}
for player in players.values():
    for key in player:
        if key == 'yaku':
            for yaku in yaku_map_short:
                yaku_key = 'yaku_' + yaku_map_short[yaku]
                value = data.get(yaku_key, list())
                value.append(player[key].get(yaku, 0))
                data[yaku_key] = value
            continue
        
        value = data.get(key, list())
        value.append(player[key])
        data[key] = value
df_players = pd.DataFrame.from_dict(data)

In [11]:
df_players.describe().T.head(50)

,count,mean,std,min,25%,50%,75%,max
player_id,57.0,511.771930,433.267157,10.0,214.0,388.0,769.0,1509.0
num_games,57.0,6.807018,0.854371,1.0,7.0,7.0,7.0,7.0
num_games_aborted,57.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
num_rounds,57.0,81.333333,12.135152,12.0,79.0,83.0,88.0,98.0
num_rounds_dealer,57.0,20.333333,4.141831,3.0,19.0,21.0,22.0,31.0
num_dealer_wins,57.0,5.175439,2.982825,1.0,3.0,5.0,6.0,15.0
num_wins,57.0,18.087719,5.029487,3.0,15.0,19.0,21.0,29.0
num_wins_ron,57.0,12.070175,4.008293,2.0,10.0,12.0,14.0,26.0
num_wins_double_ron,57.0,0.315789,0.571898,0.0,0.0,0.0,1.0,2.0
num_wins_triple_ron,57.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


### Количество раундов и различные исходы

In [12]:
outcomes = {}
for game in games:
    for round_ in game['rounds']:
        outcomes['all'] = outcomes.get('all', 0) + 1
        outcomes[round_['outcome']] = outcomes.get(round_['outcome'], 0) + 1
        if round_['outcome'] == 'ron':
            outcomes['ron_all'] = outcomes.get('ron_all', 0) + 1
        if round_['outcome'] == 'multiron':
            outcomes['ron_all'] = outcomes.get('ron_all', 0) + 1
            if round_['multi_ron'] == 2:
                outcomes['double_ron'] = outcomes.get('double_ron', 0) + 1
            if round_['multi_ron'] == 3:
                outcomes['triple_ron'] = outcomes.get('triple_ron', 0) + 1
        if round_['outcome'] == 'draw':
            if round_['tempai'] == '':
                outcomes['draw_all_noten'] = outcomes.get('draw_all_noten', 0) + 1

rounds = outcomes['all']
{outcome: (outcomes[outcome], 100 * outcomes[outcome] / rounds) for outcome in outcomes}

{'all': (1159, 100.0),
 'tsumo': (343, 29.594477998274375),
 'ron': (670, 57.80845556514237),
 'ron_all': (679, 58.584987057808455),
 'draw': (128, 11.04400345125108),
 'draw_all_noten': (2, 0.1725625539257981),
 'abort': (9, 0.7765314926660914),
 'multiron': (9, 0.7765314926660914),
 'double_ron': (9, 0.7765314926660914)}

### Яку и их процентное соотношение

In [13]:
wins = outcomes['ron'] + 2 * outcomes.get('double_ron', 0) + 3 * outcomes.get('triple_ron', 0)
df_players['yaku_yakuhai_all'] = df_players['yaku_yakuhai1'] + 2 * df_players['yaku_yakuhai2'] +\
    3 * df_players['yaku_yakuhai3'] + 4 * df_players['yaku_yakuhai4'] + 5 * df_players['yaku_yakuhai5']
columns = [col for col in df_players.columns if 'yaku_' in col]
columns.sort(reverse=True, key=lambda col: df_players[col].sum())
for col in columns:
    percent = 100 * df_players[col].sum() / wins
    if percent == 0:
        continue
    percent = round(percent, 0 if percent >= 10 else (1 if percent >= 1 else 2))
    percent = int(percent) if percent >= 10 else percent
    print(f'  <tr>')
    print(f'    <td>{yaku_short_rus[col[5:]]}</td>')
    print(f'    <td>{df_players[col].sum()}</td>')
    print(f'    <td>{percent}%</td>')
    print(f'  </tr>')
    break

  <tr>
    <td>Риичи</td>
    <td>453</td>
    <td>66%</td>
  </tr>


### Риичи-палочки

In [14]:
bets_all = df_players['yaku_riichi'].sum() + df_players['riichi_bets_lost'].sum()
bets_won = df_players['yaku_riichi'].sum()
feeds_under_riichi = df_players['num_feeds_under_riichi'].sum()
bets_all, bets_won, bets_won / bets_all, feeds_under_riichi, feeds_under_riichi / bets_all

(871, 453, 0.5200918484500574, 94, 0.1079219288174512)

In [15]:
# количество набросов в рон из-под риичи
df_players.sort_values('num_feeds_under_riichi', ascending=False).head(6)\
    [['player_id', 'player_name', 'num_feeds_under_riichi', 'yaku_riichi', 'riichi_bets_lost']]

,player_id,player_name,num_feeds_under_riichi,yaku_riichi,riichi_bets_lost
44,847,Гришина Ольга,5,7,9
35,758,Хвалюк Полина,4,7,13
32,15,Титова Елена,4,8,7
40,817,Подлипаев Сергей,4,11,16
24,218,Борзоногов Виталий,4,13,8
22,769,Чуров Петр,4,10,9


In [16]:
# количество набросов в одинарные роны из-под риичи
def count_single_ron_under_riichi(player_id: int) -> int:
    single_ron_count = 0
    for game in games:
        for round_ in game['rounds']:
            if round_['outcome'] == 'ron':
                if round_['loser_id'] == player_id and str(player_id) in (round_['riichi_bets'].split(',')):
                    single_ron_count += 1
    return single_ron_count

# {players[player_id]['player_name']: count_single_ron_under_riichi(player_id)
#    for player_id in (847, 758, 10, 15, 769)}

In [17]:
# Раздачи с тремя и четырьмя риичи на столе
bets = None
riichi_rounds = 0
cnt = Counter()
for game in games:
    for round_ in game['rounds']:
        if round_['outcome'] in ('ron', 'tsumo', 'draw'):
            if len(round_['riichi_bets'].split(',')) >= 3:
                bets = [int(x) for x in round_['riichi_bets'].split(',')]
            round_wins = [round_]
        elif round_['outcome'] == 'multiron':
            for win in round_['wins']:
                if len(win['riichi_bets'].split(',')) >= 3:
                    bets = [int(x) for x in win['riichi_bets'].split(',')]
                    break
            round_wins = round_['wins']
        if bets:
            riichi_rounds += 1 
            print(f'{riichi_rounds}. {len(bets)}: ', end='')
            for player_name in [players[x]['player_name'] for x in game['players']]:
                bet = '+' if player_name in [players[x]['player_name'] for x in bets] else ' '
                print(f'[{bet}]{player_name} ', end='')
                cnt[player_name] += 1
            print('||| ', end='')
            if round_['outcome'] == 'ron':
                print(f"ron {players[round_['loser_id']]['player_name']} -> {players[round_['winner_id']]['player_name']}")
            elif round_['outcome'] == 'tsumo':
                print(f"tsumo -> {players[round_['winner_id']]['player_name']}")
            elif round_['outcome'] == 'draw':
                print(f"draw")
            elif round_['outcome'] == 'multiron':
                winners = [players[win['winner_id']]['player_name'] for win in round_['wins']]
                print(f"multiron {players[round_['loser_id']]['player_name']} -> {winners}")

            if 'replay_link' in game and game['replay_link']:
                print(' ' * 8 + f"Реплей: {game['replay_link']}", end='')
                #print(f"&tw={game['players'].index(round_wins[0]['winner_id'])}&ts={game['rounds'].index(round_)}")                      
                print(f"&tw={game['players'].index(316 if 316 in game['players'] else round_wins[0]['winner_id'])}&ts={game['rounds'].index(round_)}")                      

        bets = None
cnt.most_common(6)

1. 3: [ ]Матвеева Алена [+]Седнева Наталия [+]Мишаков Дмитрий [+]Федоркевич Антон ||| ron Федоркевич Антон -> Седнева Наталия
        Реплей: http://tenhou.net/0/?log=2019072821gm-0009-16324-81a295d0&tw=2&ts=5
2. 3: [+]Колпаков Денис [ ]Пинигин Игорь [+]Мишаков Дмитрий [+]Dekabrev Yan ||| ron Мишаков Дмитрий -> Колпаков Денис
        Реплей: http://tenhou.net/0/?log=2019072820gm-0009-16324-b7a88b58&tw=2&ts=7
3. 3: [+]Масуи Хидэ [+]Богданов Владимир [ ]Ким Олег [+]Петрова Анна ||| ron Петрова Анна -> Богданов Владимир
        Реплей: http://tenhou.net/0/?log=2019072820gm-0009-16324-61b61805&tw=1&ts=1
4. 3: [+]Клочков Андрей [ ]Мишаков Дмитрий [+]Титов Артем [+]Соляков Павел ||| ron Мишаков Дмитрий -> Соляков Павел
        Реплей: http://tenhou.net/0/?log=2019072819gm-0009-16324-849bd7b4&tw=1&ts=10
5. 3: [+]Клочков Андрей [+]Мишаков Дмитрий [ ]Титов Артем [+]Соляков Павел ||| tsumo -> Клочков Андрей
        Реплей: http://tenhou.net/0/?log=2019072819gm-0009-16324-849bd7b4&tw=1&ts=12
6. 3

[('Мишаков Дмитрий', 4),
 ('Матвеева Алена', 2),
 ('Федоркевич Антон', 2),
 ('Колпаков Денис', 2),
 ('Богданов Владимир', 2),
 ('Ким Олег', 2)]

### По соотношению рон/цумо и набросов
Имя | Раздач | Рон | Цумо | Набросов | % выигрыша | % набросов

Сортировка по % выигрыша

In [18]:
df_players['wins_perc'] = round(100 * df_players['num_wins'] / df_players['num_rounds'], 1)
df_players['feed_perc'] = round(100 * df_players['num_feeds'] / df_players['num_rounds'], 1)
for index, row in df_players.sort_values(by='wins_perc', ascending=False).iterrows():
    url = f"https://gui.mjtop.net/eid{event_id}/user/{row['player_id']}?l=ru"
    color = '#dfd' if row['num_wins'] > row['num_feeds'] else '#fdd'
    print(f'  <tr style="background-color: {color};">')
    print(f"    <td><a href=\"{url}\">{row['player_name']}</a></td>")
    print(f"    <td>{row['num_rounds']}</td>")
    print(f"    <td>{row['num_wins_ron']}</td>")
    print(f"    <td>{row['num_wins_tsumo']}</td>")
    print(f"    <td>{row['num_feeds']}</td>")
    print(f"    <td>{row['wins_perc']}%</td>")
    print(f"    <td>{row['feed_perc']}%</td>")
    print(f'  </tr>')
    break

  <tr style="background-color: #dfd;">
    <td><a href="https://gui.mjtop.net/eid204/user/879?l=ru">Зыкова Софья</a></td>
    <td>82</td>
    <td>26</td>
    <td>3</td>
    <td>10</td>
    <td>35.4%</td>
    <td>12.2%</td>
  </tr>


### Количество не сыгранных полностью ханчанов

In [19]:
df_players[['player_id', 'player_name', 'num_games', 'num_games_aborted', 'num_rounds']]\
    .sort_values('num_rounds', ascending=False).iloc[np.r_[0:5, -5:0]]

,player_id,player_name,num_games,num_games_aborted,num_rounds
25,1473,Лойко Алексей,7,0,98
32,15,Титова Елена,7,0,96
3,271,Титов Артем,7,0,95
17,55,Лисихина Людмила,7,0,94
4,315,Луговкин Михаил,7,0,90
54,272,Титова Анна,6,0,70
53,327,Игрок замены 2,6,0,66
55,326,Игрок замены 1,6,0,63
56,1083,Степаненко Сергей,5,0,53
52,331,Игрок замены 4,1,0,12


In [20]:
df_players['num_wins_and_feeds'] = df_players['num_wins'] + df_players['num_feeds']
df_players[['player_id', 'player_name', 'num_rounds', 'num_wins', 'wins_perc', 'num_wins_ron', 'num_wins_tsumo', 'num_feeds', 'feed_perc', 'num_wins_and_feeds']]\
    .sort_values('feed_perc', ascending=False).iloc[np.r_[0:5, -5:0]]
    #.sort_values('num_wins_and_feeds', ascending=False).iloc[np.r_[0:5, -5:0]]


,player_id,player_name,num_rounds,num_wins,wins_perc,num_wins_ron,num_wins_tsumo,num_feeds,feed_perc,num_wins_and_feeds
26,949,Косолапов Никита,82,20,24.4,13,7,20,24.4,40
49,110,Седнева Наталия,85,27,31.8,17,10,20,23.5,47
44,847,Гришина Ольга,77,17,22.1,10,7,18,23.4,35
35,758,Хвалюк Полина,89,22,24.7,16,6,20,22.5,42
27,514,Янц Александра,82,19,23.2,10,9,18,22.0,37
55,326,Игрок замены 1,63,11,17.5,6,5,5,7.9,16
21,70,Ермолаев Андрей,81,15,18.5,10,5,6,7.4,21
11,773,Шашкин Юрий,82,13,15.9,6,7,5,6.1,18
9,45,Лебедева Елена,83,19,22.9,14,5,5,6.0,24
19,810,Клочков Андрей,88,17,19.3,13,4,5,5.7,22


In [21]:
(df_players['num_games_aborted'].sum() / 4, df_players['num_games'].sum() / 4, 
 df_players['num_games_aborted'].sum() / df_players['num_games'].sum())

(0.0, 97.0, 0.0)

### Занятое место в зависимости от стартового ветра

In [22]:
winds = ['East', 'South', 'West', 'North']
distr = {wind: {1: 0, 2: 0, 3: 0, 4: 0} for wind in winds}
for game in games:
    for idx, player_id in enumerate(game['players']):
        distr[winds[idx]][game['final_results'][str(player_id)]['place']] += 1 / len(games)
for wind in distr:
    distr[wind]['avg'] = sum([place * distr[wind][place] for place in distr[wind]])
distr

{'East': {1: 0.2577319587628865,
  2: 0.2783505154639175,
  3: 0.22680412371134015,
  4: 0.2371134020618556,
  'avg': 2.4432989690721643},
 'South': {1: 0.17525773195876287,
  2: 0.29896907216494845,
  3: 0.22680412371134015,
  4: 0.29896907216494845,
  'avg': 2.6494845360824737},
 'West': {1: 0.30927835051546393,
  2: 0.16494845360824742,
  3: 0.24742268041237106,
  4: 0.2783505154639175,
  'avg': 2.494845360824742},
 'North': {1: 0.2577319587628865,
  2: 0.2577319587628865,
  3: 0.29896907216494845,
  4: 0.18556701030927833,
  'avg': 2.412371134020618}}

### Таблица выплат

Имя | Выигрыш | Выплат по рон | Выплат по цумо | Чужие риичи | доры

Сортировка по выигрышу

In [23]:
df_players['gains_all'] = df_players['gains_ron'] + df_players['gains_tsumo']
for index, row in df_players.sort_values(by='gains_all', ascending=False).iterrows():
    url = f"https://gui.mjtop.net/eid{event_id}/user/{row['player_id']}?l=ru"
    gains = row['gains_all']
    feed_ron = row['losses_ron']
    feed_tsumo = row['losses_tsumo']
    color = '#dfd' if gains >= feed_ron else '#fdd'
    print(f'  <tr style="background-color: {color};">')
    print(f"    <td><a href=\"{url}\">{row['player_name']}</a></td>")
    print(f"    <td>{gains}</td>")
    print(f"    <td>{feed_ron}</td>")
    print(f"    <td>{feed_tsumo}</td>")
    print(f"    <td>{row['riichi_bets_wins']}</td>")
    print(f"    <td>{row['doras']}</td>")
    print(f'  </tr>')
    break

  <tr style="background-color: #dfd;">
    <td><a href="https://gui.mjtop.net/eid204/user/110?l=ru">Седнева Наталия</a></td>
    <td>208700</td>
    <td>143200</td>
    <td>27500</td>
    <td>12</td>
    <td>38</td>
  </tr>


### Статистика по якухаям

In [24]:
df_players[['player_id', 'player_name'] + [c for c in df_players.columns if 'yakuhai' in c]]\
    .sort_values('yaku_yakuhai_all', ascending=False).iloc[np.r_[0:5, -5:0]]

,player_id,player_name,yaku_yakuhai1,yaku_yakuhai2,yaku_yakuhai3,yaku_yakuhai4,yaku_yakuhai5,yaku_yakuhai_all
8,473,Ермолаев Алексей,9,4,0,0,0,17
49,110,Седнева Наталия,4,5,0,0,0,14
48,507,Пинигин Игорь,9,2,0,0,0,13
35,758,Хвалюк Полина,9,2,0,0,0,13
6,36,Порошин Семен,7,3,0,0,0,13
2,570,Курмаев Ренат,1,1,0,0,0,3
20,16,Фалько Ирина,3,0,0,0,0,3
52,331,Игрок замены 4,1,0,0,0,0,1
1,14,Соколов Михаил,1,0,0,0,0,1
56,1083,Степаненко Сергей,1,0,0,0,0,1


### Статистика по дилерам

In [25]:
df_players['gains_dealer'] = df_players['gains_ron_dealer'] + df_players['gains_tsumo_dealer'] 
df_players[['player_id', 'player_name'] + [c for c in df_players.columns if 'dealer' in c]]\
    .sort_values('num_rounds_dealer', ascending=False).iloc[np.r_[0:5, -5:0]]
    #.sort_values('num_tsumo_feeds_dealer', ascending=False).iloc[np.r_[0:5, -5:0]]
    #.sort_values('num_dealer_wins', ascending=False).iloc[np.r_[0:5, -5:0]]


,player_id,player_name,num_rounds_dealer,num_dealer_wins,num_tsumo_feeds_dealer,gains_ron_dealer,gains_tsumo_dealer,losses_tsumo_dealer,gains_dealer
25,1473,Лойко Алексей,31,15,6,80800,36400,17300,117200
28,879,Зыкова Софья,28,13,4,57400,13400,10600,70800
17,55,Лисихина Людмила,26,10,3,53100,8000,4300,61100
24,218,Борзоногов Виталий,26,12,6,84200,23500,17000,107700
49,110,Седнева Наталия,25,10,1,61200,41700,4100,102900
20,16,Фалько Ирина,16,1,4,3900,0,14200,3900
54,272,Титова Анна,15,2,2,4000,0,12100,4000
55,326,Игрок замены 1,14,1,4,2900,0,12600,2900
56,1083,Степаненко Сергей,11,1,3,3900,0,11300,3900
52,331,Игрок замены 4,3,1,1,0,7700,4000,7700


### Выигрыш по очкам

In [26]:
df_players['avg_gains_all'] = df_players['gains_all'] / df_players['num_wins'] 
df_players[['player_id', 'player_name', 'num_rounds', 'num_wins'] + [c for c in df_players.columns if 'gains' in c]]\
    .sort_values('gains_all', ascending=False).iloc[np.r_[0:5, -5:0]]
    #.sort_values('num_tsumo_feeds_dealer', ascending=False).iloc[np.r_[0:5, -5:0]]
    #.sort_values('num_dealer_wins', ascending=False).iloc[np.r_[0:5, -5:0]]


,player_id,player_name,num_rounds,num_wins,gains_ron,gains_tsumo,gains_ron_dealer,gains_tsumo_dealer,gains_tempai,gains_all,gains_dealer,avg_gains_all
49,110,Седнева Наталия,85,27,122100,86600,61200,41700,4500,208700,102900,7729.629630
25,1473,Лойко Алексей,98,27,121500,61500,80800,36400,7000,183000,117200,6777.777778
24,218,Борзоногов Виталий,88,25,116100,60900,84200,23500,1500,177000,107700,7080.000000
50,290,Петрущенко Сергей,79,22,86600,84100,25700,5800,4000,170700,31500,7759.090909
6,36,Порошин Семен,89,23,89100,75200,40800,11600,8000,164300,52400,7143.478261
54,272,Титова Анна,70,10,52800,17700,4000,0,2500,70500,4000,7050.000000
55,326,Игрок замены 1,63,11,36900,28400,2900,0,5500,65300,2900,5936.363636
20,16,Фалько Ирина,86,10,54000,6200,3900,0,5000,60200,3900,6020.000000
56,1083,Степаненко Сергей,53,5,25100,12000,3900,0,4500,37100,3900,7420.000000
52,331,Игрок замены 4,12,3,13500,7700,0,7700,0,21200,7700,7066.666667


### Выплаты по рон и цумо

In [27]:
df_players['losses_all'] = df_players['losses_ron'] + df_players['losses_tsumo'] 
df_players['avg_losses_ron'] = df_players['losses_ron'] / df_players['num_feeds'] 
df_players[['player_id', 'player_name', 'num_rounds', 'num_feeds'] + [c for c in df_players.columns if 'losses' in c]]\
    .sort_values('losses_all', ascending=False).iloc[np.r_[0:5, -10:0]]

,player_id,player_name,num_rounds,num_feeds,losses_ron,losses_tsumo,losses_tsumo_dealer,losses_noten,losses_all,avg_losses_ron
49,110,Седнева Наталия,85,20,143200,27500,4100,2500,170700,7160.000000
35,758,Хвалюк Полина,89,20,121600,48400,28300,7000,170000,6080.000000
32,15,Титова Елена,96,19,113400,41000,5500,7000,154400,5968.421053
43,321,Петрова Анна,82,18,129900,23200,1800,0,153100,7216.666667
41,23,Губанова Кристина,89,19,112600,36800,10100,10000,149400,5926.315789
5,388,Колпаков Денис,88,8,40900,35300,13500,10000,76200,5112.500000
53,327,Игрок замены 2,66,6,30600,44300,14000,9000,74900,5100.000000
28,879,Зыкова Софья,82,10,38800,33100,10600,5000,71900,3880.000000
8,473,Ермолаев Алексей,86,9,39000,32500,17100,5500,71500,4333.333333
6,36,Порошин Семен,89,9,35000,34200,17600,3000,69200,3888.888889


### Выплаты по ничьим

In [28]:
df_players['draw_balance'] = df_players['gains_tempai'] - df_players['losses_noten'] 
df_players['avg_draw_balance'] = df_players['draw_balance'] / df_players['num_draws'] 
df_players[['player_id', 'player_name', 'gains_tempai', 'losses_noten'] + [c for c in df_players.columns if 'draw' in c]]\
    .sort_values('num_draws', ascending=False).iloc[np.r_[0:5, -5:0]]

,player_id,player_name,gains_tempai,losses_noten,num_draws,num_draws_tempai,num_draws_noten,draw_balance,avg_draw_balance
9,45,Лебедева Елена,6000,23500,18,5,13,-17500,-972.222222
20,16,Фалько Ирина,5000,14500,15,4,11,-9500,-633.333333
46,12,Мешин Виталий,3500,19000,15,3,12,-15500,-1033.333333
19,810,Клочков Андрей,5000,16500,15,4,11,-11500,-766.666667
11,773,Шашкин Юрий,8500,9500,14,6,8,-1000,-71.428571
39,1479,Кульгаева Варвара,4500,3000,5,3,2,1500,300.000000
49,110,Седнева Наталия,4500,2500,5,3,2,2000,400.000000
54,272,Титова Анна,2500,3000,5,3,2,-500,-100.000000
24,218,Борзоногов Виталий,1500,1500,3,2,1,0,0.000000
52,331,Игрок замены 4,0,1500,1,0,1,-1500,-1500.000000


### Риичи-палки

In [29]:
df_players['num_damaten'] = df_players['num_wins'] - (df_players['num_wins_open_hand']\
                          + df_players['yaku_riichi'] + df_players['yaku_daburu_riichi'])
df_players['wins_open_hand_perc'] = round(100 * df_players['num_wins_open_hand'] / df_players['num_wins'], 2)
df_players['damatan_perc'] = round(100 * df_players['num_damaten']\
                                      / (df_players['num_wins'] - df_players['num_wins_open_hand']), 2)
df_players['riichi_efficiency'] = round(100 * (df_players['yaku_riichi'] + df_players['yaku_daburu_riichi'])\
        / (df_players['yaku_riichi'] + df_players['yaku_daburu_riichi'] + df_players['riichi_bets_lost']), 2)
df_players['num_wins_closed_hand'] = df_players['num_wins'] - df_players['num_wins_open_hand']
df_players[['player_id', 'player_name', 'num_wins', 'num_wins_closed_hand', 'num_wins_open_hand', 'wins_open_hand_perc',\
            'num_damaten', 'damatan_perc', 'yaku_riichi', 'yaku_daburu_riichi', 'riichi_bets_wins', 'riichi_bets_lost', 'riichi_efficiency']]\
    .sort_values('riichi_bets_wins', ascending=False).iloc[np.r_[0:5, -5:0]].T

,28,18,13,0,49,14,6,53,56,52
player_id,879,214,1507,27,110,378,36,327,1083,331
player_name,Зыкова Софья,Богданов Владимир,Jean-Michel MORISSE,Федоркевич Антон,Седнева Наталия,Ким Олег,Порошин Семен,Игрок замены 2,Степаненко Сергей,Игрок замены 4
num_wins,29,16,22,21,27,21,23,14,5,3
num_wins_closed_hand,11,13,6,7,14,11,12,11,4,2
num_wins_open_hand,18,3,16,14,13,10,11,3,1,1
wins_open_hand_perc,62.07,18.75,72.73,66.67,48.15,47.62,47.83,21.43,20,33.33
num_damaten,1,2,5,0,-1,2,6,1,0,0
damatan_perc,9.09,15.38,83.33,0,-7.14,18.18,50,9.09,0,0
yaku_riichi,10,11,1,7,15,9,6,10,4,2
yaku_daburu_riichi,0,0,0,0,0,0,0,0,0,0


### Количество дор

In [30]:
df_players['avg_doras'] = round(df_players['doras'] / df_players['num_wins'], 2)
df_players[['player_id', 'player_name'] + [c for c in df_players.columns if 'dora' in c]]\
    .sort_values('avg_doras', ascending=False).iloc[np.r_[0:5, -10:0]]

,player_id,player_name,doras,doras_normal,doras_uradora,doras_kandora,doras_kanuradora,doras_akadora,avg_doras
1,14,Соколов Михаил,28,28,0,0,0,0,2.33
56,1083,Степаненко Сергей,10,10,0,0,0,0,2.00
52,331,Игрок замены 4,6,6,0,0,0,0,2.00
6,36,Порошин Семен,43,43,0,0,0,0,1.87
8,473,Ермолаев Алексей,43,43,0,0,0,0,1.79
15,1508,Dekabrev Yan,24,24,0,0,0,0,1.00
34,705,Ельченко Юлия,15,15,0,0,0,0,1.00
7,770,Лавникевич Сергей,21,21,0,0,0,0,0.95
23,662,Амелевич Максим,19,19,0,0,0,0,0.95
5,388,Колпаков Денис,16,16,0,0,0,0,0.94


In [31]:
df_players['doras'].sum(), df_players['doras'].sum() / df_players['num_wins'].sum()

(1361, 1.3200775945683803)

### Самые дорогие руки

In [32]:
least_han, least_yaku, least_dora = 0, 7, 0
baimans = Counter()
for game in games:
    for round_ in game['rounds']:
        if round_['outcome'] in ('ron', 'tsumo'):
            round_wins = [round_]
        elif round_['outcome'] == 'multiron':
            round_wins = round_['wins']
        else:
            continue
        for win in round_wins:
            if win['dora'] >= least_dora and \
               win['han'] >= least_han and \
               win['han'] - win['dora'] >= least_yaku:
                winner = players[win['winner_id']]['player_name']
                loser_id = win.get('loser_id', round_.get('loser_id', 0))
                loser = players[loser_id]['player_name'] if loser_id else '(цумо)'
                dealer = players[get_dealer_id(round_['round_index'], game)]['player_name']
                is_dealer = '(дилер) ' if winner == dealer else ''
                yaku_list = [yaku_map_rus[int(yaku)].lower() for yaku in win['yaku'].split(',')]
                yaku_str = ', '.join(yaku_list)
                print(f"{winner} {is_dealer}from {loser}, {win['han']} хан: {yaku_str}, дора {win['dora']}")
                if 'replay_link' in game and game['replay_link']:
                    print(' ' * 8 + f"Реплей: {game['replay_link']}", end='')
                    print(f"&tw={game['players'].index(win['winner_id'])}&ts={game['rounds'].index(round_)}")
                baimans[winner] += 1
    #if baimans:
    #    break
baimans#.most_common(8)

Колпаков Денис from Мишаков Дмитрий, 8 хан: риичи, шосанген, хоницу, якухай 2, дора 0
        Реплей: http://tenhou.net/0/?log=2019072820gm-0009-16324-b7a88b58&tw=0&ts=7
Федоркевич Антон from (цумо), 7 хан: тойтой, хоницу, якухай 3, дора 0
        Реплей: http://tenhou.net/0/?log=2019072819gm-0009-16324-e32f8f6d&tw=1&ts=0
Титов Артем (дилер) from (цумо), 8 хан: риичи, мендзен цумо, пин-фу, иипейко, саншоку, таняо, дора 1
        Реплей: http://tenhou.net/0/?log=2019072817gm-0009-16324-6f269616&tw=1&ts=3
Хвалюк Полина from (цумо), 8 хан: риичи, иппацу, мендзен цумо, пин-фу, саншоку, таняо, дора 1
        Реплей: http://tenhou.net/0/?log=2019072817gm-0009-16324-32afe874&tw=2&ts=6


Counter({'Колпаков Денис': 1,
         'Федоркевич Антон': 1,
         'Титов Артем': 1,
         'Хвалюк Полина': 1})

### Якуманы

In [33]:
yakumans = Counter()
for game in games:
    for round_ in game['rounds']:
        if round_['outcome'] in ('ron', 'tsumo'):
            round_wins = [round_]
        elif round_['outcome'] == 'multiron':
            round_wins = round_['wins']
        else:
            continue
        for win in round_wins:
            if win['han'] < 0:
                winner = players[win['winner_id']]['player_name']
                loser_id = win.get('loser_id', round_.get('loser_id', 0))
                loser = players[loser_id]['player_name'] if loser_id else '(цумо)'
                dealer = players[get_dealer_id(round_['round_index'], game)]['player_name']
                is_dealer = '(дилер) ' if winner == dealer else ''
                yaku_list = [yaku_map_rus[int(yaku)] for yaku in win['yaku'].split(',')]
                yaku_str = ', '.join(yaku_list)
                print(f"{winner} {is_dealer}from {loser}, {win['han']} хан: {yaku_str}, {win['dora']} дор")
                if 'replay_link' in game and game['replay_link']:
                    print(' ' * 8 + f"Реплей: {game['replay_link']}", end='')
                    print(f"&tw={game['players'].index(win['winner_id'])}&ts={game['rounds'].index(round_)}")
                yakumans[winner] += 1
    if baimans:
        break
yakumans.most_common(8)

[]

### Самые разнообразные руки

In [34]:
different_yakus = 5
for game in games:
    for round_ in game['rounds']:
        if round_['outcome'] in ('ron', 'tsumo'):
            round_wins = [round_]
        elif round_['outcome'] == 'multiron':
            round_wins = round_['wins']
        else:
            continue
        for win in round_wins:
            yaku_list = [yaku_map_rus[int(yaku)] for yaku in win['yaku'].split(',')]
            if len(yaku_list) >= different_yakus:
                winner = players[win['winner_id']]['player_name']
                loser_id = win.get('loser_id', round_.get('loser_id', 0))
                loser = players[loser_id]['player_name'] if loser_id else '(цумо)'
                dealer = players[get_dealer_id(round_['round_index'], game)]['player_name']
                is_dealer = '(дилер) ' if winner == dealer else ''
                yaku_str = ', '.join(yaku_list)
                print(f"{len(yaku_list)} яку: {winner} {is_dealer}from {loser}, {win['han']} хан: {yaku_str}, {win['dora']} дор")
                if 'replay_link' in game and game['replay_link']:
                    print(' ' * 8 + f"Реплей: {game['replay_link']}", end='')
                    print(f"&tw={game['players'].index(win['winner_id'])}&ts={game['rounds'].index(round_)}")
                baimans[winner] += 1
    #if baimans:
    #    break

5 яку: Матвеева Алена from (цумо), 6 хан: Риичи, Иппацу, Мендзен цумо, Пин-фу, Саншоку, 0 дор
        Реплей: http://tenhou.net/0/?log=2019072822gm-0009-16324-199f5249&tw=0&ts=12
5 яку: Масуи Хидэ from (цумо), 9 хан: Риичи, Мендзен цумо, Пин-фу, Саншоку, Таняо, 3 дор
        Реплей: http://tenhou.net/0/?log=2019072822gm-0009-16324-b54bc49a&tw=2&ts=1
5 яку: Суслова Анастасия (дилер) from (цумо), 5 хан: Риичи, Иппацу, Мендзен цумо, Пин-фу, Таняо, 0 дор
        Реплей: http://tenhou.net/0/?log=2019072817gm-0009-16324-97c6a6da&tw=1&ts=2
6 яку: Титов Артем (дилер) from (цумо), 8 хан: Риичи, Мендзен цумо, Пин-фу, Иипейко, Саншоку, Таняо, 1 дор
        Реплей: http://tenhou.net/0/?log=2019072817gm-0009-16324-6f269616&tw=1&ts=3
6 яку: Хвалюк Полина from (цумо), 8 хан: Риичи, Иппацу, Мендзен цумо, Пин-фу, Саншоку, Таняо, 1 дор
        Реплей: http://tenhou.net/0/?log=2019072817gm-0009-16324-32afe874&tw=2&ts=6
5 яку: Губанова Кристина from Степаненко Сергей, 6 хан: Риичи, Иппацу, Пин-фу, Саншоку

In [35]:
print(f"{winner} {is_dealer}from {loser}, {win['han']} хан: {yaku_str}, {win['dora']} дор")

Губанова Кристина from Степаненко Сергей, 3 хан: Риичи, Иппацу, Пин-фу, Саншоку, Таняо, 0 дор


### Количество различных комбинаций

In [36]:
unique_yaku = [c for c in df_players.columns if 'yaku_' in c and (not 'yakuhai' in c or 'yakuhai_' in c)]
df_players['unique_yaku'] = df_players[unique_yaku].astype(bool).sum(axis=1)
df_players[['player_id', 'player_name', 'unique_yaku']]\
    .sort_values('unique_yaku', ascending=False).iloc[np.r_[0:6, -5:0]]

,player_id,player_name,unique_yaku
5,388,Колпаков Денис,15
32,15,Титова Елена,14
10,756,Рожкова Анастасия,12
23,662,Амелевич Максим,12
12,1509,Моторин Александр,12
31,65,Суслова Анастасия,12
36,1086,Потапов Павел,6
19,810,Клочков Андрей,6
56,1083,Степаненко Сергей,6
52,331,Игрок замены 4,5
